In [1]:
import subscriptionManager as sm
import feedbackManager as fm

from datetime import datetime, timedelta

from IPython.display import display, clear_output
from ipywidgets import Layout, Button, Box, VBox, HBox, Output, Dropdown, Label, HTML, Text, RadioButtons

import matplotlib.pyplot as plt
import numpy as np

matrix = []
rentalMatrix = []

today = datetime.today()
subs = sm.load_subscriptions()

with open('Music_Info.txt', 'r') as f:
    f.readline()
    for line in f:
        s = line.strip()
        matrix.append(list(s.split(',')))

with open('Rental.txt', 'r') as r:
    r.readline()
    for line in r:
        s = line.strip()
        rentalMatrix.append(list(s.split(',')))

def not_available(ID):
    for rental in rentalMatrix:
        if rental[0] == ID and rental[2] == "":
            return True
    return False

In [2]:
def searchByArtist(artist):
    found = False 
    for z in range(len(matrix)):
        if matrix[z][1] == artist:
            found = True
            if not_available(matrix[z][0]):
                availability = "Unavailable"
            else:
                availability = "Available"
            print(f">> Record ID: {matrix[z][0]}\nArtist: {matrix[z][1]}\nTitle: {matrix[z][2]}\nMedium: {matrix[z][3]}\nGenre: {matrix[z][4]}\nAvailability: {availability}\n")
    if not found:
        print(f"Artist Not Found")


def searchByTitle(title):
        found = False 
        for z in range(len(matrix)):
            if matrix[z][2] == title:
                found = True
                if not_available(matrix[z][0]):
                    availability = "Unavailable"
                else:
                    availability = "Available"
                print(f">> Record ID: {matrix[z][0]}\nArtist: {matrix[z][1]}\nTitle: {matrix[z][2]}\nMedium: {matrix[z][3]}\nGenre: {matrix[z][4]}\nAvailability: {availability}\n")   
        if not found:
            print(f"Title Not Found")


def searchByMedium(medium):
        found = False 
        for z in range(len(matrix)):
            if matrix[z][3] == medium:
                found = True
                if not_available(matrix[z][0]):
                    availability = "Unavailable"
                else:
                    availability = "Available"
                print(f">> Record ID: {matrix[z][0]}\nArtist: {matrix[z][1]}\nTitle: {matrix[z][2]}\nMedium: {matrix[z][3]}\nGenre: {matrix[z][4]}\nAvailability: {availability}\n")
        if not found:
            print(f"Medium Not Found")


def searchByGenre(genre):
        found = False 
        for z in range(len(matrix)):
            if matrix[z][4] == genre:
                found = True
                if not_available(matrix[z][0]):
                    availability = "Unavailable"
                else:
                    availability = "Available"
                print(f">> Record ID: {matrix[z][0]}\nArtist: {matrix[z][1]}\nTitle: {matrix[z][2]}\nMedium: {matrix[z][3]}\nGenre: {matrix[z][4]}\nAvailability: {availability}\n")
        if not found:
            print(f"Genre Not Found.")

def artistClicked(b):
    output.clear_output()
    with output:
        try:
            artist = searchBar.value.strip()
            if not artist:
                print("Error: Invalid Search")
                return
            searchByArtist(artist)

        except Exception as e:
            print(f"Error processing search request: {str(e)}")
        
    
def titleClicked(b):
    output.clear_output()
    with output:
        try:
            title = searchBar.value.strip()
            if not title:
                print(f"Error: Invalid Search")
                return
            searchByTitle(title)

        except Exception as e:
            print(f"Error processing search request: {str(e)}")

def mediumClicked(b):
    output.clear_output()
    with output:
        try:
            medium = searchBar.value.strip()
            if not medium:
                print(f"Error: Invalid Search")
                return
            searchByMedium(medium)

        except Exception as e:
            print(f"Error processing search request: {str(e)}")
    
def genreClicked(b):
    output.clear_output()
    with output:
        try:
            genre = searchBar.value.strip()
            if not genre:
                print(f"Error: Invalid Search")
                return
            searchByGenre(genre)

        except Exception as e:
            print(f"Error processing search request: {str(e)}")

button1 = Button(description="Search By Artist...")
button2 = Button(description="Search By Title...")
button3 = Button(description="Search By Medium...")
button4 = Button(description="Search By Genre...")
    
buttons_hbox = HBox([button1, button2, button3, button4])
buttons_hbox.layout = Layout(align_items='center', justify_content='space-between', width='100%')
    
searchBar = Text(
    description='Search Bar:',
    placeholder='Type Here...',
    layout=Layout(width='290px', margin='0px 0px 20px 0px')
)
    
header = HTML(
    value="<h1 style='text-align: center; font-size: 14px; font-weight: bold;'>Search For Records:</h1>"
)
    
output = Output()
    
vbox = VBox([searchBar])
vbox.layout = Layout(align_items='center', justify_content='center', margin='50px auto', width='100%')
    
button1.on_click(artistClicked)
button2.on_click(titleClicked)
button3.on_click(mediumClicked)
button4.on_click(genreClicked)

def searchMenu():
    vbox2 = VBox([output])
    vbox2.layout = Layout(align_items='center', justify_content='flex-end', margin='50px auto 20px 70px', width='90%')
    ui = VBox([header, vbox, buttons_hbox, vbox2])
    display(ui)

In [3]:
def renting(customerID, recordID):
    if customerID not in subs:
        print(f"Customer Does Not Have A Valid Subscription.")
        return

    found = False
    for z in range(len(matrix)):
        if matrix[z][0] == recordID:
            found = True
    if not found:
        print(f"Record Not Found.")
        return
            

    sub = subs[customerID]
    startDate = sub["StartDate"]
    endDate = sub["EndDate"]
    subType = sub["SubscriptionType"]
    if subType == "Basic":
        limit = 2
    else:
        limit = 7

    if not (startDate <= today <= endDate):
        print(f"This Customer's Subscription Is Not Currently Active.")
        return

    activeRentals = sum(1 for rental in rentalMatrix if rental[3] == customerID and rental[2] == "")

    if activeRentals >= limit:
        print(f"Customer Has Reached The Maximum Rental Number.")
        return

    for z in range(len(matrix)):
        if matrix[z][0] == recordID:
            if not_available(matrix[z][0]):
                availability = "Unavailable"
            else:
                availability = "Available"

    if availability == "Unavailable":
        print(f"This Record Is Unavailable To Rent Right Now.")
        return

    else: 
        date = today.strftime("%Y-%m-%d")
        with open("Rental.txt", "a") as f:
            f.write(f"{recordID},{date},,{customerID}\n")
    print(f"Successfully Rented Out")

def submit_clicked(b):
    output1.clear_output()
    with output1:
        try:
            customerID = customer_id_box.value.strip()
            if not customerID:
                print(f"Error: Customer ID cannot be empty")
                return
                
            recordID = record_id_box.value
            if not recordID:
                print(f"Error: Record ID cannot be empty")
                return
                
            
            renting(customerID, recordID)

        except Exception as e:
            print(f"Error processing rent request: {str(e)}")

customer_id_box = Text(placeholder='Enter Customer ID...', description="Customer ID:")
record_id_box = Text(placeholder='Enter Record ID...', description="Record ID:")
rentHeader = HTML(
        value="<h1 style='text-align: left; font-size: 14px; font-weight: bold;'>Rent Record:</h1>"
    )

submit_button1 = Button(description="Rent Record", button_style='success')
output1 = Output()

submit_button1.on_click(submit_clicked)

def rentMenu():
    ui = VBox([
        rentHeader,
        customer_id_box,
        record_id_box,
        submit_button1,
        output1
    ])
    display(ui)

In [4]:
def returning(recordID, starRating, comments):
    found = False
        
    for z in range(len(matrix)):
        if matrix[z][0] == recordID:
            found = True
            if not_available(matrix[z][0]):  
                availability = "Unavailable"
            else:
                availability = "Available"
            break  
    
    if not found:
        print(f"Record ID '{recordID}' Not Found.")
        return
        
    if availability == "Available":
        print(f"Record ID '{recordID}' Is Not Returnable")
        return


    else:
        date = today.strftime("%Y-%m-%d")
        for rental in rentalMatrix:
            if rental[0] == recordID and rental[2] == "":
                rental[2] = date
                break
        
        with open("Rental.txt", "r") as f:
            lines = f.readlines()

        header = lines[0] if lines else ""  
        
        with open("Rental.txt", "w") as f:
            f.write(header)
            for rental in rentalMatrix:
                f.write(",".join(map(str, rental)) + "\n")
                
        if comments == "":
            fm.add_feedback(recordID, starRating,"", "Music_Feedback.txt")
        else: 
            fm.add_feedback(recordID, starRating, comments, "Music_Feedback.txt")

        print(f"Returned Successfully")
       

def submitClicked(b):
    output2.clear_output()
    with output2:
        try:
            recordID = record_id_box.value.strip()
            if not recordID:
                print(f"Error: Record ID cannot be empty")
                return
                
            starRating = rating_buttons.value
            if not starRating:
                print(f"Error: Please select a star rating")
                return
                
            comments = comments_box.value.strip()
            
            returning(recordID, starRating, comments)

        except Exception as e:
            print(f"Error processing return: {str(e)}")



record_id_box = Text(placeholder='Enter Record ID...', description="Record ID:")
rating_buttons = RadioButtons(
    options=['1', '2', '3', '4', '5'],
    description='Rating:',
    layout=Layout(width='auto')
)
comments_box = Text(placeholder='Enter Comments (optional)...', description="Comments:")
submit_button = Button(description="Return Record", button_style='success')
returnHeader = HTML(
        value="<h1 style='text-align: left; font-size: 14px; font-weight: bold;'>Return Record:</h1>"
    )
output2 = Output()

submit_button.on_click(submitClicked)

def returnMenu():
    ui = VBox([
        returnHeader,
        record_id_box,
        rating_buttons,
        HBox([comments_box]),
        submit_button,
        output2
    ])
    display(ui)

In [5]:
def plotSuggestions(suggestions, graph_output, reset=False):
    if not suggestions:
        with graph_output:
            clear_output(wait=True)
            print(f"No suggestions to plot.")
        return

    titles = [s[1] for s in suggestions]
    days = [s[3] for s in suggestions]

    x = np.arange(len(titles))
    
    with graph_output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(10, 7))
        bar_width = 2.0

        bars = ax.bar(
            x,
            days,
            color=plt.cm.viridis(np.linspace(0.3, 0.7, len(titles))),
            edgecolor='gray',
            linewidth=0.8
        )
        for bar in bars:
            height = bar.get_height()
            ax.text(
                bar.get_x() + bar.get_width() / 2,
                height + 10, 
                f"{height}",
                ha='center',
                va='bottom',
                fontsize=10,
                color='black'
            )

        ax.set_title("Unpopular Records", fontsize=16, fontweight='bold', color='navy', pad=20)
        ax.set_xlabel("Records", fontsize=14, labelpad=10)
        ax.set_ylabel("Days Since Last Rental", fontsize=14, labelpad=10)
        ax.set_xticks(x)
        ax.set_xticklabels(titles, rotation=45, ha='right', fontsize=10)
        ax.yaxis.grid(True, linestyle='--', alpha=0.7)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        plt.tight_layout()
        plt.show()
    

def pruningSuggestions(matrix, rentalMatrix):
    inactiveDate = datetime.today() - timedelta(days=365)
    
    suggestions = []
    
    for record in matrix:
        recordID = record[0]
        title = record[2]  
        returnDate = record[2] 

        if returnDate == "": 
            continue

        recordRentals = [rental for rental in rentalMatrix if rental[0] == recordID]

        lastReturnDate = None
        itemStillOut = False

        for rental in recordRentals:
            if rental[2] == "":
                itemStillOut = True  
                continue  

            try:
                returnDate = datetime.strptime(rental[2], '%Y-%m-%d')
            except ValueError:
                print(f"Invalid date found for Record ID {recordID}: {rental[2]}")
                continue

            if not lastReturnDate or returnDate > lastReturnDate:
                lastReturnDate = returnDate

        if itemStillOut:
            continue

        if lastReturnDate:
            daysSinceReturn = (datetime.today() - lastReturnDate).days

            if daysSinceReturn >= 365:
                suggestions.append((recordID, title, lastReturnDate.strftime('%Y-%m-%d'), daysSinceReturn))
    
    if suggestions:
        for suggestion in suggestions:
            return suggestions

def removingRecord(recordID, matrix):
    for i, record in enumerate(matrix):
        if record[0] == recordID:
            del matrix[i] 

    with open("Music_Info.txt", "r") as f:
            lines = f.readlines()

    header = lines[0] if lines else ""

    with open("Music_Info.txt", "w") as f:
        f.write(header)
        for record in matrix:
            f.write(",".join(map(str, record)) + "\n")
            
    
def removeButtonClick(b, recordDropdown, matrix, graph_output, suggestions, output):
    recordID = recordDropdown.value
    if recordID == "":
        with output:
            clear_output(wait = True)
            print(f"Please Select A Record")
    else:
        removingRecord(recordID, matrix)
        with output:
            clear_output(wait = True)
            print(f"Record Successfully Removed")
            updateDropdownOptions(recordDropdown, matrix, rentalMatrix)
            updateBarChart(graph_output, suggestions)

def updateDropdownOptions(recordDropdown, matrix, rentalMatrix):
    suggestions = pruningSuggestions(matrix, rentalMatrix)
    if not suggestions:
        recordDropdown.options = [('Select', '')]
        return
    recordDropdown.options = [('Select', '')] + [(f"{s[1]} (ID: {s[0]}, Last Rented: {s[2]}, {s[3]} Days Ago)", s[0]) for s in suggestions]

def updateBarChart(graph_output, suggestions, reset=False):
    suggestions = pruningSuggestions(matrix, rentalMatrix)
    if not suggestions:
        plotSuggestions(suggestions, graph_output, reset=True)
        return
    plotSuggestions(suggestions, graph_output, reset=False)
    
        
def interactivePruning(matrix, rentalMatrix):
    suggestions = pruningSuggestions(matrix, rentalMatrix)
    if not suggestions:
        print(f"No pruning suggestions available.")
        return
    
    recordDropdown = Dropdown(
        options=[('Select', '')] + [(f"{s[1]} (ID: {s[0]}, Last Rented: {s[2]}, {s[3]} Days Ago)", s[0]) for s in suggestions],
        description='Suggestions:',
        style={'description_width': 'initial'},
        layout=Layout(width='80%')
    )
    
    removeButton = Button(
        description="Remove Selected Record",
        button_style='danger',
        icon='trash',
        layout=Layout(width='80%')
    )

    pruningHeader = HTML(
        value="<h1 style='text-align: left; font-size: 18px; font-weight: ;'>Welcome To Inventory Pruning!</h1>"
    )

    graph_output = Output()
    plotSuggestions(suggestions, graph_output)
    
    output = Output()

    removeButton.on_click(lambda b: removeButtonClick(b, recordDropdown, matrix, graph_output, suggestions, output))
    display(pruningHeader, graph_output, VBox([recordDropdown, removeButton, output]))

In [6]:
def search_button_clicked(b, output):
    with output:
        clear_output(wait = True)
        searchMenu()

def rent_button_clicked(b, output):
     with output:
        clear_output(wait = True)
        rentMenu()

def return_button_clicked(b, output):
     with output:
        clear_output(wait = True)
        returnMenu()

def pruning_button_clicked(b, output):
     with output:
        clear_output(wait = True)
        interactivePruning(matrix, rentalMatrix)

def menu():
    items_layout = Layout(width='auto')
    
    box_layout = Layout(
        display='flex',
        flex_flow='column',
        align_items='stretch',
        border='solid',
        width='100%',
        height='400px',
    )
    
    
    header = HTML(
        value="""
        <div style="
        display: flex; 
        justify-content: center; 
        align-items: center; 
        width: 100%;">
        <h1 style="
        text-align: center; 
        font-size: 44px; 
        font-weight: bold; 
        color: white; 
        background-color: #333333; 
        padding: 10px 20px; 
        display: inline-block; 
        border-radius: 15px; 
        box-shadow: 0px 4px 15px rgba(0, 0, 0, 0.2);
        font-family: 'Comic Sans MS', monospace;
        ">
        MENU
        </h1>
        </div>
        """
    )
    
    output = Output()
    
    buttons = [
    Button(description="Search For Music", 
           layout=Layout(width='99.6%', height='100px'), 
           style={'button_color': '#D3D3D3'}), 
    Button(description="Rent Music", 
           layout=Layout(width='99.6%', height='100px'), 
           style={'button_color': '#D3D3D3'}),
    Button(description="Return Music", 
           layout=Layout(width='99.6%', height='100px'), 
           style={'button_color': '#D3D3D3'}),
    Button(description="Inventory Pruning Suggestions", 
           layout=Layout(width='99.6%', height='100px'), 
           style={'button_color': '#D3D3D3'}) 
    ]
    
    buttons[0].on_click(lambda b: search_button_clicked(b, output))
    buttons[1].on_click(lambda b: rent_button_clicked(b, output))
    buttons[2].on_click(lambda b: return_button_clicked(b, output))
    buttons[3].on_click(lambda b: pruning_button_clicked(b, output))
    
    box = Box(children=buttons, layout=box_layout)
    return VBox([header, box, output])

menu()
